# Traveling Salesman Problem Implementation in Google ORTools, Google Maps Distance Matrix API, and Google Earth/Maps KML Exportation
## Conor Falvey, 2022

In [484]:
# Imports
import os
import ortools
import pandas as pd
import numpy as np
import urllib.parse
import requests
from pathlib import Path
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [487]:
# Configs
# Jupyter/Pandas display params
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

# Pathing, Requests, Map Stuff, etc.
CSV_FOLDER = Path(os.getcwd() + "/data")
BASE_URL = "https://maps.googleapis.com/maps/api/distancematrix"
OUTPUT = "json"
API_KEY = ""
START = "San Francisco"
DEST_LIMIT = 25

In [98]:
# Import all csv files from data
# This is now not very needed as all 3 files have
# been concatenated together.
def import_places(path: Path):
    places = []
    for path in path.glob('*.csv'):
        places.append(pd.read_csv(path))
    df = pd.concat(places)[["X", "Y", "Name"]]
    return df

In [335]:
# You can either move around the order of the CSV
# file to determine the start and skip this, but
# I started added formatting here so I kept it.
def define_start(df: pd.DataFrame):
    # Get row index of starting name
    start_row = df[df["Name"] == START]
    # Drop row, add to index -1, increment index,
    # sort, and reset indices as a hacky way to
    # force an element to the top
    df = df.drop(start_row.index)
    df.loc[-1] = start_row.iloc[0]
    df.index = df.index + 1
    df.sort_index(inplace=True)
    df = df.reset_index(drop=True)
    # Casting for combining coords
    df["X"] = df["X"].astype(str)
    df["Y"] = df["Y"].astype(str)
    df["Coords"] = df[['Y', 'X']].agg(','.join, axis=1)
    return df

In [489]:
# The Google Maps Distance Matrix API takes nxm origins
# and destinations. To simplify things, we iterate over
# singular origins and feed in all possible destinations.
# The endpoint has a 25 destination limit per request, so
# we paginate and join them before adding them into the 
# distance matrix. If no route is found, we set the distance
# to 999999999 in the next method so we can easily find them 
# later and print out information about its coordinates to 
# manually double check in Google maps and update the coordinates.
def req_distance_matrix(df: pd.DataFrame):
    length = df.shape[0]
    matrix = np.empty((length, length), dtype=int)
    for i, r in df.iterrows():
        print(f"Calculating Routes from {r['Name']}")
        origin = r["Coords"]
        raw_dests = df["Coords"]
        distances = []
        for chunk in [raw_dests[i:i + DEST_LIMIT] for i in range(0, len(raw_dests), DEST_LIMIT)]:
            dests = "|".join(chunk)
            dists = get_maps_distance(origin, dests)
            if 999999999 in dists:
                print(f"{i}    {dists.index(999999999)}    {chunk.iloc[0]}    {dests}")
            distances.extend(dists)
        matrix[i] = distances
    return matrix

In [490]:
# Format the URL into the endpoints expectations and
# check it completed ok. If no route is found, the 
# status of that element will be 'NOT_FOUND', so we 
# check for an OK code.
def get_maps_distance(origin: str, dest: str):
    url = format_url(origin, dest)
    req = requests.get(url)
    if not req.ok:
        raise ValueError
    dists = []
    for x in req.json()["rows"][0]["elements"]:
        if x["status"] == "OK":
            dists.append(x["distance"]["value"])
        else:
            dists.append(999999999)
    return dists

In [491]:
# URL encode the URL and format together
def format_url(origin: str, dest: str):
    format_origin = urllib.parse.quote_plus(origin)
    format_dest = urllib.parse.quote_plus(dest)
    return f"{BASE_URL}/{OUTPUT}?origins={format_origin}&destinations={format_dest}&key={API_KEY}"

In [450]:
# Main method to import DF, format it,
# and compute the distance matrix. It
# takes 1183 requents to compute everything 
# with destination limits on the endpoint
def main():
    df = import_places(CSV_FOLDER)
    df = define_start(df)
    matrix = req_distance_matrix(df)
    return df, matrix

In [443]:
d, m = main()

Calculating Routes from San Francisco
Calculating Routes from New York
Calculating Routes from Los Angeles
Calculating Routes from Chicago
Calculating Routes from Houston
Calculating Routes from Philadelphia
Calculating Routes from San Antonio
Calculating Routes from San Diego
Calculating Routes from Dallas
Calculating Routes from San Jose
Calculating Routes from Jacksonville
Calculating Routes from Fort Worth
Calculating Routes from Charlotte
Calculating Routes from Seattle
Calculating Routes from Washington
Calculating Routes from El Paso
Calculating Routes from Portland
Calculating Routes from Las Vegas
Calculating Routes from Detroit
Calculating Routes from Memphis
Calculating Routes from Louisville
Calculating Routes from Baltimore
Calculating Routes from Milwaukee
Calculating Routes from Albuquerque
Calculating Routes from Tucson
Calculating Routes from Fresno
Calculating Routes from Kansas City
Calculating Routes from Mesa
Calculating Routes from Omaha
Calculating Routes from Co

In [493]:
# Setup data and router for TSP in ORTools
data = {}
data['distance_matrix'] = m
data['num_vehicles'] = 1
data['depot'] = 0

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [494]:
# Returns distance between two locations given the 
# distance matrix above
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [495]:
# Standard Solution Printer from documentation with added
# list tracking so we can index into the dataframe from 
# earlier and print out an ordered list of locations on the route
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} meters'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    idx_list = []
    while not routing.IsEnd(index):
        idx = manager.IndexToNode(index)
        idx_list.append(idx)
        plan_output += ' {} ->'.format(idx)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    idx_list.append(manager.IndexToNode(index))
    plan_output += 'Route distance: {} meters\n'.format(route_distance)
    print(plan_output)
    return idx_list

In [497]:
# General setup of callbacks and routing algorithms
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

# Solve away!
solution = routing.SolveWithParameters(search_parameters)
if solution:
    l = print_solution(manager, routing, solution)

Objective: 36902198 meters
Route for vehicle 0:
 0 -> 45 -> 75 -> 161 -> 160 -> 159 -> 106 -> 16 -> 157 -> 116 -> 158 -> 13 -> 156 -> 95 -> 155 -> 154 -> 81 -> 113 -> 150 -> 149 -> 148 -> 147 -> 146 -> 145 -> 29 -> 39 -> 76 -> 144 -> 119 -> 143 -> 142 -> 110 -> 141 -> 103 -> 140 -> 139 -> 92 -> 34 -> 84 -> 28 -> 96 -> 85 -> 26 -> 94 -> 55 -> 129 -> 82 -> 118 -> 22 -> 3 -> 130 -> 83 -> 66 -> 91 -> 18 -> 63 -> 138 -> 42 -> 60 -> 54 -> 107 -> 101 -> 114 -> 137 -> 88 -> 98 -> 90 -> 108 -> 77 -> 1 -> 57 -> 99 -> 5 -> 78 -> 89 -> 21 -> 14 -> 135 -> 115 -> 31 -> 102 -> 59 -> 56 -> 12 -> 109 -> 134 -> 10 -> 53 -> 32 -> 133 -> 132 -> 68 -> 40 -> 79 -> 72 -> 80 -> 131 -> 136 -> 117 -> 104 -> 51 -> 44 -> 86 -> 20 -> 128 -> 111 -> 19 -> 74 -> 127 -> 93 -> 41 -> 87 -> 4 -> 46 -> 71 -> 6 -> 112 -> 11 -> 38 -> 70 -> 8 -> 58 -> 35 -> 37 -> 105 -> 69 -> 126 -> 124 -> 123 -> 15 -> 125 -> 100 -> 23 -> 121 -> 120 -> 73 -> 27 -> 64 -> 61 -> 122 -> 24 -> 62 -> 7 -> 52 -> 50 -> 43 -> 30 -> 166 -> 2 -> 48 -> 

In [505]:
# Print out ordered list of locations
# that we can implement the route for in 
# Google Maps and Google Earth to plot
for idx, el in enumerate(l):
    print(f"{idx}:\t{d.iloc[el]['Name']}")

0:	San Francisco
1:	Stockton
2:	Sacramento
3:	Lassen Volcanic National Park
4:	Redwood National and State Parks
5:	Crater Lake National Park
6:	Salem
7:	Portland
8:	Mount Rainier National Park
9:	Olympia
10:	Olympic National Park
11:	Seattle
12:	Glacier National Park
13:	Helena
14:	Yellowstone National Park
15:	Grand Teton National Park
16:	Boise
17:	Salt Lake City
18:	Capitol Reef National Park
19:	Arches National Park
20:	Canyonlands National Park
21:	Mesa Verde National Park
22:	Black Canyon of the Gunnison National Park
23:	Great Sand Dunes National Park and Preserve
24:	Colorado Springs
25:	Aurora
26:	Denver
27:	Rocky Mountain National Park
28:	Cheyenne
29:	Wind Cave National Park
30:	Badlands National Park
31:	Pierre
32:	Theodore Roosevelt National Park
33:	Bismarck
34:	Voyageurs National Park
35:	Isle Royale National Park
36:	Saint Paul
37:	Minneapolis
38:	Des Moines
39:	Omaha
40:	Lincoln
41:	Topeka
42:	Kansas City
43:	Jefferson City
44:	St. Louis
45:	Gateway Arch National Park
